In [1]:
import warnings
warnings.filterwarnings('ignore')
import argparse
import numpy as np
import networkx as nx
#import node2vec
from gensim.models import Word2Vec
import random

In [2]:

def read_graph(input,weighted,directed):
    '''
    Reads the input network in networkx.
    '''
    # 权重图
    if weighted:
        G = nx.read_edgelist(input, nodetype=int, data=(('weight', float),), create_using=nx.DiGraph())
    # 无权图
    else:
        G = nx.read_edgelist(input, nodetype=int, create_using=nx.DiGraph())
        for edge in G.edges():
            G[edge[0]][edge[1]]['weight'] = 1
    # 无向操作
    if not directed:
        G = G.to_undirected()

    return G

In [3]:
nx_G = read_graph('../Graph/karate.edgelist',False,False)

## RandomWalk

In [15]:

class Graph():
    # 出事化设置参数
    def __init__(self, nx_G, is_directed):
        self.G = nx_G
        self.is_directed = is_directed
       
    
    def deep_walk(self, walk_length, start_node):
        '''
        Simulate a random walk starting from start node.
        '''
        G = self.G
        # 上一步计算出的alias table，完成O(1)的采样
        alias_nodes = self.alias_nodes

        walk = [start_node]

        #  直到生成长度为walk_length的节点序列位为止
        while len(walk) < walk_length:
            cur = walk[-1]
            # 对邻居节点排序，目的是和alias table计算时的顺序对应起来
            cur_nbrs = sorted(G.neighbors(cur))
            if len(cur_nbrs) > 0:
                # 节点序列只有一个节点的情况
                walk.append(cur_nbrs[alias_draw(alias_nodes[cur][0], alias_nodes[cur][1])])
               
            else:
                break

        return walk

    def simulate_walks(self, num_walks, walk_length):
        '''
        Repeatedly simulate random walks from each node.
        '''
        G = self.G
        walks = []
        nodes = list(G.nodes())
        print ('Walk iteration:')
        for walk_iter in range(num_walks):
            print (str(walk_iter+1), '/', str(num_walks))
            # 打乱节点顺序
            random.shuffle(nodes)
            for node in nodes:
                # node2vec_walk是一次有偏的随机游走
                walks.append(self.deep_walk(walk_length=walk_length, start_node=node))

        return walks


    def preprocess_transition_probs(self):
        '''
        Preprocessing of transition probabilities for guiding the random walks.
        '''
        G = self.G
        is_directed = self.is_directed

        alias_nodes = {}
        # 节点概率alias sampling和归一化
        for node in G.nodes():
            unnormalized_probs = [G[node][nbr]['weight'] for nbr in sorted(G.neighbors(node))]
            norm_const = sum(unnormalized_probs)
            normalized_probs =  [float(u_prob)/norm_const for u_prob in unnormalized_probs]
            alias_nodes[node] = alias_setup(normalized_probs)
            # 信息展示
            if node == 2:
                print (unnormalized_probs)
                print (norm_const)
                print (normalized_probs)
                print (alias_nodes[node])

    
        
        self.alias_nodes = alias_nodes
    
        return





In [16]:

def alias_setup(probs):
    '''
    Compute utility lists for non-uniform sampling from discrete distributions.
    Refer to https://hips.seas.harvard.edu/blog/2013/03/03/the-alias-method-efficient-sampling-with-many-discrete-outcomes/
    for details
    '''
    # 总过K个长度
    K = len(probs)
    # q个0
    q = np.zeros(K)
    J = np.zeros(K, dtype=np.int)

    smaller = []
    larger = []
    
    # 将各个概率分成两组，一组的概率值大于1，另一组的概率值小于1
    for kk, prob in enumerate(probs):
        q[kk] = K*prob
        if q[kk] < 1.0:
            smaller.append(kk)
        else:
            larger.append(kk)
    
    # 使用贪心算法，将概率值小于1的不断填满
    # pseudo code step 3
    while len(smaller) > 0 and len(larger) > 0:
        small = smaller.pop()
        large = larger.pop()

        J[small] = large
        # 更新概率值
        q[large] = q[large] + q[small] - 1.0
        if q[large] < 1.0:
            smaller.append(large)
        else:
            larger.append(large)

    return J, q

In [17]:

def alias_draw(J, q):
    '''
    Draw sample from a non-uniform discrete distribution using alias sampling.
    '''
    K = len(J)

    kk = int(np.floor(np.random.rand()*K))
    # 取自己 
    if np.random.rand() < q[kk]:
        return kk
    # 取alias table存的节点
    else:
        return J[kk]


In [18]:

directed = False
G = Graph(nx_G, directed)


In [19]:
G.preprocess_transition_probs()


[1, 1, 1, 1, 1, 1, 1, 1, 1]
9
[0.1111111111111111, 0.1111111111111111, 0.1111111111111111, 0.1111111111111111, 0.1111111111111111, 0.1111111111111111, 0.1111111111111111, 0.1111111111111111, 0.1111111111111111]
(array([0, 0, 0, 0, 0, 0, 0, 0, 0]), array([1., 1., 1., 1., 1., 1., 1., 1., 1.]))


In [20]:
num_walks = 10
walk_length = 20
# 有偏的随机游走生成节点序列
walks = G.simulate_walks(num_walks, walk_length)
# 展示一个节点序列的长度
print (len(walks[0]))

Walk iteration:
1 / 10
2 / 10
3 / 10
4 / 10
5 / 10
6 / 10
7 / 10
8 / 10
9 / 10
10 / 10
20


## 训练embedding

In [21]:
def learn_embeddings(walks,dimensions,window_size,workers,iter):
    '''
    Learn embeddings by optimizing the Skipgram objective using SGD.
    '''
    # 将node的类型int转化为string
    # walks = [map(str, walk) for walk in walks]
    walk_lol = []
    for walk in walks:
        tmp = []
        for node in walk:
            tmp.append(str(node))
        walk_lol.append(tmp)
    # 调用gensim包运行word2vec
    model = Word2Vec(walk_lol, size=dimensions, window=window_size, min_count=0, sg=1, workers=workers,
                     iter=iter)
    # model.save_word2vec_format(args.output)
    # 保存embedding信息
#     model.wv.save_word2vec_format(args.output)

    return model


In [22]:
model = learn_embeddings(walks,16,3,-1,10)
print ('finished')

finished


In [23]:

from scipy import spatial
def cos_similarity(v1, v2):
    return 1 - spatial.distance.cosine(v1, v2)

# # 相似节点组1
print (cos_similarity(model['17'], model['6']))
print (cos_similarity(model['7'], model['6']))
print (cos_similarity(model['7'], model['5']))


# # 相似节点组2
# print (cos_similarity(model['34'], model['33']))
# print (cos_similarity(model['34'], model['9']))
# print (cos_similarity(model['34'], model['31']))

# # 不相似节点组
# print (cos_similarity(model['17'], model['25']))
# print (cos_similarity(model['7'], model['25']))


0.1182999536395073
-0.049709271639585495
0.011760308407247066


In [24]:

# k-means聚类
from sklearn import  cluster
from sklearn.metrics import adjusted_rand_score
from sklearn.model_selection import train_test_split
import pandas as pd
embedding_node=[]
for i in range(1,35):
    j=str(i)
    embedding_node.append(model[j])
embedding_node=np.matrix(embedding_node).reshape((34,-1))
y_pred = cluster.KMeans(n_clusters=3, random_state=9).fit_predict(embedding_node) # 调用 test_RandomForestClassifier
y_pred



array([0, 0, 0, 2, 1, 1, 1, 2, 1, 2, 1, 2, 0, 1, 2, 1, 2, 0, 1, 0, 1, 2,
       0, 1, 2, 0, 0, 2, 1, 0, 2, 1, 0, 1], dtype=int32)